In [4]:
import requests
import os

import tensorflow as tf
import resnest

SCORE_CAM_SOURCE = 'https://raw.githubusercontent.com/haofanwang/Score-CAM/master/images'
KERAS_VIS_SOURCE = 'https://raw.githubusercontent.com/keisen/tf-keras-vis/master/examples/images'

class config:
    class data:
        image_size = (512, 512)
        path = './data'
        images = os.path.join(path, 'misc')
        sources = [
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00000073.JPEG',
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00000091.JPEG',
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00000198.JPEG',
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00000476.JPEG',
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00002193.JPEG',

            f'{KERAS_VIS_SOURCE}/goldfish.jpg',
            f'{KERAS_VIS_SOURCE}/bear.jpg',
            f'{KERAS_VIS_SOURCE}/soldiers.jpg',

            'https://3.bp.blogspot.com/-W__wiaHUjwI/Vt3Grd8df0I/AAAAAAAAA78/7xqUNj8ujtY/s400/image02.png',
            'http://www.aviationexplorer.com/Diecast_Airplanes_Aircraft/delta_Airbus_diecast_airplane.jpg',
            'https://www.petcare.com.au/wp-content/uploads/2017/09/Dalmatian-2.jpg',
            'http://sites.psu.edu/siowfa15/wp-content/uploads/sites/29639/2015/10/dogcat.jpg',
            'https://consciouscat.net/wp-content/uploads/2009/08/multiple-cats-300x225.jpg',
            'https://images2.minutemediacdn.com/image/upload/c_crop,h_843,w_1500,x_0,y_78/f_auto,q_auto,w_1100/v1554995977/shape/mentalfloss/iStock-157312120.jpg',
            'http://www.reptilefact.com/wp-content/uploads/2016/08/Images-of-San-Francisco-Garter-Snake.jpg',
        ]

def load_images():
    global ds, class_names, file_names

    os.makedirs(config.data.images, exist_ok=True)

    file_names = [os.path.join(config.data.images, os.path.basename(f)) for f in config.data.sources]
    class_names = [os.path.splitext(os.path.basename(f))[0] for f in config.data.sources]
    
    for s, d in zip(config.data.sources, file_names):
        if not os.path.exists(d):
            download_image(s, d)
        
    ds = tf.data.Dataset.from_tensor_slices((file_names,))
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).cache()
    ds = ds.apply(tf.data.experimental.ignore_errors(log_warning=True))
    ds = ds.batch(len(file_names))

    return next(iter(ds))

def download_image(url, path):
    r = requests.get(url, allow_redirects=True)
    with open(path, 'wb') as f:
        f.write(r.content)
    return path

@tf.function
def load_image(x):
    x = tf.io.read_file(x)
    x = tf.image.decode_jpeg(x, channels=3)
    x = tf.image.resize(x, config.data.image_size)
    return x

@tf.function
def augment_fn(image, y):
    image = tf.image.random_brightness(image, .2)
    image = tf.image.random_contrast(image, .75, 1.)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    return image, y

x = load_images()

2022-06-12 12:02:12.481883: W tensorflow/core/kernels/data/experimental/ignore_errors_dataset_op.cc:117] Error raised with error message Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{function_node __inference_load_image_52}}{{node DecodeJpeg}}]]
2022-06-12 12:02:12.482000: W tensorflow/core/kernels/data/experimental/ignore_errors_dataset_op.cc:117] Error raised with error message Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{function_node __inference_load_image_52}}{{node DecodeJpeg}}]]
2022-06-12 12:02:12.482017: W tensorflow/core/kernels/data/experimental/ignore_errors_dataset_op.cc:117] Error raised with error message Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{function_node __inference_load_image_52}}{{node DecodeJpeg}}]]
2022-06-12 12:02:12.487878: W tensorflow/core/kernels/data/experimental/ignore_errors_dataset_op.cc:117] Error raised with error message Unknown image file format. One of JPEG, PNG, GIF, BMP

In [5]:
from keras.applications.imagenet_utils import preprocess_input

x_ = preprocess_input(x, mode='torch')

In [18]:
model = resnest.ResNeSt101(input_shape=(256, 256, 3), weights='imagenet')

In [10]:
tf.reduce_min(x), tf.reduce_max(x)

(<tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=255.0>)

In [14]:
outputs = model.predict(tf.image.resize(x_, (256, 256), method='bilinear'), verbose=0)

2022-06-12 12:04:04.123173: I tensorflow/compiler/xla/service/service.cc:170] XLA service 0x7f83280081d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-12 12:04:04.123247: I tensorflow/compiler/xla/service/service.cc:178]   StreamExecutor device (0): Host, Default Version
2022-06-12 12:04:05.841784: I tensorflow/compiler/jit/xla_compilation_cache.cc:478] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [15]:
predictions = tf.keras.applications.imagenet_utils.decode_predictions(outputs, top=3)
print(
    *(
        ' | '.join(f'{name.rjust(15)} {conf:.2%}' for _id, name, conf in p)
        for p in predictions
    ),
    sep='\n'
)

           kite 71.17% | sulphur-crested_cockatoo 0.17% |   ruffed_grouse 0.17%
     white_wolf 81.98% |     timber_wolf 3.73% |        red_wolf 0.29%
         bulbul 84.13% |          toucan 0.35% |       bee_eater 0.23%
         magpie 58.56% |    black_grouse 6.00% | European_gallinule 3.40%
        obelisk 48.48% |        pedestal 25.62% |          beacon 5.25%
golden_retriever 82.51% | Labrador_retriever 1.04% |  cocker_spaniel 0.81%
       airliner 76.39% |            wing 3.48% |        warplane 0.23%
      dalmatian 74.63% |          kuvasz 0.29% | German_short-haired_pointer 0.12%
          tabby 33.82% |            chow 14.10% |       tiger_cat 5.52%
     spider_web 56.50% |     barn_spider 14.03% |   garden_spider 7.63%
   garter_snake 24.92% |      sidewinder 21.67% |    horned_viper 20.46%


In [32]:
model2 = resnest.ResNeSt101(
    input_shape=(512, 512, 3),
    weights='imagenet',
    include_top=False,
    pooling=False,
    dilation=4,
)

In [33]:
model2.output

<KerasTensor: shape=(None, 64, 64, 2048) dtype=float32 (created by layer 'stage4_block3_shorcut_act')>

In [34]:
import numpy as np

for wa, wb in zip(model.weights[:-2], model2.weights):
    np.testing.assert_array_almost_equal(wa.numpy(), wb.numpy())